In [1]:
import numpy as np

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Span

output_notebook()


Loading BokehJS ...

In [2]:
# reload 

from importlib import reload

import IntLib
import AutoDiff
import GenFuncs

reload(IntLib)
reload(AutoDiff)
reload(GenFuncs)


<module 'GenFuncs' from 'c:\\Users\\k1m19\\OneDrive\\Desktop\\Numerical\\IntLib\\GenFuncs.py'>

In [3]:
from IntLib import Interval as IA

from AutoDiff import (
    AutoDiff as AD,
    _ensure_AD as ensure_AD,
)

from GenFuncs import (
    mag,
    mig,
    sqrt,
    sin,
    cos,
    mid,
    rad,
    exp,
    log,
    atan,
)



# TODO
- tests
- deal with empty set
- extened division ?
- numpy arrays...

In [6]:
f = lambda x: -2.001 + 3 * x - x**3

fig = figure(width=600, height=300)
x = np.linspace(-3, 3)
y, dy = f(AD(x, 1))
fig.line(x, y)
fig.line(x, dy, color='red')
fig.add_layout(Span(location=0, dimension='width'))
show(fig)

# Extended IA broken

# <span style="color:red"> TODO </span>

In [5]:
dbg = True

def prn(*x):
    global dbg
    if dbg: 
        print(*x)

def Nx(f, x):
    xx = ensure_AD(x, 1)  # variable
    fx = f(xx)
    mx = IA(mid(x))
    nx = mx - f(mx) / fx.dx
    return nx

acc = []

def interval_newton_ext(f, x, tol, prev=0, left=None):
    global acc
    l = prev
    l += 1
    x0 = x
    left = "L" if left else "R"
    prn(f"\n{(prev,l,left)}: x0: {x0}")
    if x0.isempty:
        prn(f"{(prev,l,left)} empty")
        return
    while rad(x) > tol:
        nx = Nx(f, x)
        prn(f"{(prev,l,left)}: nx: {nx}")
        if nx.is_extended:
            xl, xr = (x & IA(-np.inf, nx.x_)), (x & IA(nx._x, np.inf))
            interval_newton_ext(f, xl, tol, l, True)
            interval_newton_ext(f, xr, tol, l, False)
            return
        x = x & nx
        prn(f"{(prev,l,left)}: x: {x} {x.rad:+g}")
        if x.isempty:
            prn(f"{(prev,l,left)}: no roots {x}")
            return
    if (x in x0) and (f(x).has_zero) and (not (Nx(f, x) & x).isempty):
        maybe = "maybe " if x == x0 else ""
        print(f"{(prev,l,left)}: {maybe}ℝ f({x})={f(x)}")
        acc.append(x)
    else:
        prn(f"{(prev,l,left)}: no roots {x}")


f = lambda x: -2.001 + 3 * x - x**3

interval_newton_ext(f, IA(-3, +3), 0.1)
acc


(0, 1, 'R'): x0: [-3.₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀, +3.₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀]
(0, 1, 'R'): nx: [-∞, -₀.₀83375₀₀₀₀₀₀₀₀₀₀₀]∪[+₀.66699999999999993, +∞]

(1, 2, 'L'): x0: [-3.₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀, -₀.₀83375₀₀₀₀₀₀₀₀₀₀₀]
(1, 2, 'L'): nx: [-∞, -1.665₀94963₀8427929]∪[-₀.54751692732723711, +∞]

(2, 3, 'L'): x0: [-3.₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀, -1.665₀94963₀8427929]
(2, 3, 'L'): nx: [-2.1787₀456478₀59493, -1.6382₀914456321187]
(2, 3, 'L'): x: [-2.1787₀456478₀59493, -1.665₀94963₀8427929] +0.256805
(2, 3, 'L'): nx: [-2.₀4747853₀66389318, -1.9813₀947359721432]
(2, 3, 'L'): x: [-2.₀4747853₀66389318, -1.9813₀947359721432] +0.0330845
(2, 3, 'L'): ℝ f([-2.₀4747853₀66389318, -1.9813₀947359721432])=[-₀.36563242343139724, +₀.638446239₀39192₀6]

(2, 3, 'R'): x0: [-₀.54751692732723711, -₀.₀83375₀₀₀₀₀₀₀₀₀₀₀]
(2, 3, 'R'): nx: [+₀.66334₀992₀4536945, +1.₀7265461512881544]
(2, 3, 'R'): x: ∅ +nan
(2, 3, 'R'): no roots ∅

(1, 2, 'R'): x0: [+₀.66699999999999993, +3.₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀]
(1, 2, 'R'): nx: [-∞, +1.72249₀927276₀4181]∪[+3.43331₀81583

[[-2.₀4747853₀66389318, -1.9813₀947359721432],
 [+₀.932199937₀8924753, +1.₀4465₀6₀3₀8515589],
 [+1.₀77588₀3187763₀3₀, +1.174₀43₀71418₀2255]]

In [ ]:
# f = lambda x: sin(x)*(x+cos(x))
f = lambda x: -2.001 + 3 * x - x**3
l, dbg = 0, False
acc = []
interval_newton_ext(f, IA(-3, 3), 0.1)
acc

(2, 3, 'L'): ℝ f([-2.₀4747853₀66389318, -1.9813₀947359721432])=[-₀.36563242343139724, +₀.638446239₀39192₀6]
(5, 6, 'L'): maybe ℝ f([+₀.932199937₀8924753, +1.₀4465₀6₀3₀8515589])=[-₀.344422₀459₀6976₀₀, +₀.322873119₀1519211]
(5, 6, 'R'): maybe ℝ f([+1.₀77588₀3187763₀3₀, +1.174₀43₀71418₀2255])=[-₀.38651₀₀278₀625654, +₀.26983833814738256]


[[-2.₀4747853₀66389318, -1.9813₀947359721432],
 [+₀.932199937₀8924753, +1.₀4465₀6₀3₀8515589],
 [+1.₀77588₀3187763₀3₀, +1.174₀43₀71418₀2255]]

In [ ]:
f = lambda x: x**2-4*x+3
l = 0
dbg = True
acc = []
interval_newton_ext(f, IA(0, 4), 0.01)
acc


(0, 1, 'R'): x0: [+₀.₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀, +4.₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀]
(0, 1, 'R'): nx: [+1.5₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀, +1.5₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀]
(0, 1, 'R'): x: [+1.5₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀, +1.5₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀] +0
(0, 1, 'R'): no roots [+1.5₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀, +1.5₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀]


[]

# Why there is no 1 as a root?

In [ ]:
x = np.linspace(-3, 4, 100)

fig = figure(width=800, height=400)
y, dy = f(AD(x, 1))
fig.line(x, y)
fig.line(x, dy, color='red')
fig.add_layout(Span(location=0, dimension='width', line_color='green'))
for r in acc:
    _r, r_ = r
    fig.segment(x0=_r, y0=0, x1=r_, y1=0, color='red', line_width=10)
show(fig)
